# Inspecting Raw Data, Metrics and Calibration Solutions

In [2]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import hera_qm as hqm
import hera_cal as hc
import aipy
from pyuvdata import UVCal, UVData, UVFITS
import os
import sys
import glob
from memory_profiler import memory_usage as mu
import uvtools as uvt

## Setup Env Vars


In [3]:
# get data location
data_path = os.environ['DATA_PATH']
calfile = os.environ['CALFILE']

# get redundant info
aa = aipy.cal.get_aa('hsa7458_v001', np.array([.150]))
info = hc.omni.aa_to_info(aa)
red_bls = np.array(info.get_reds())
ants = np.unique(np.concatenate(red_bls))
Nants = len(ants)
Nside = int(np.ceil(np.sqrt(Nants)))
Yside = int(np.ceil(float(Nants)/Nside))

## Raw Autocorrelations

In [4]:
# get xx and yy files
file_index = 0
xxfiles = glob.glob("{0}/zen.2457555.5*.xx.HH.uvc".format(data_path))
yyfiles = glob.glob("{0}/zen.2457555.5*.yy.HH.uvc".format(data_path))

xxfile = xxfiles[file_index]
yyfile = yyfiles[file_index]
xxfile_base = os.path.basename(xxfile)
yyfile_base = os.path.basename(yyfile)

# Load data
uvd_xx = UVData()
uvd_xx.read_miriad(xxfile)
uvd_yy = UVData()
uvd_yy.read_miriad(yyfile)

# Get metadata
freqs = uvd_xx.freq_array.squeeze() / 1e6
times = uvd_xx.time_array.reshape(uvd_xx.Nbls, uvd_xx.Ntimes)[0, :]
Nfreqs = len(freqs)
Ntimes = len(times)

Altitude is not present in Miriad file, using known location values for HERA.
antenna number 112 has visibilities associated with it, but it has a position of (0,0,0)
antenna_diameters is not set. Using known values for HERA.


In [ ]:
# plot autos
t_index = 0
jd = times[t_index]
xlim = (-50, Nfreqs+50)
ylim = (1e-2, 1e3)

fig, axes = plt.subplots(Yside, Nside, figsize=(14,14), dpi=75)
fig.subplots_adjust(wspace=0.2, hspace=0.2)
fig.suptitle("JD = {0}".format(jd))
fig.tight_layout(rect=(0, 0, 1, 0.95))

k = 0
for i in range(Yside):
    for j in range(Nside):
        ax = axes[i,j]
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)
        if k < Nants:
            px, = ax.plot(np.abs(uvd_xx.get_data((ants[k], ants[k]))[t_index]), color='steelblue', alpha=0.5, linewidth=3)
            py, = ax.plot(np.abs(uvd_yy.get_data((ants[k], ants[k]))[t_index]), color='darkorange', alpha=0.5, linewidth=3)
            ax.set_yscale('log')
            ax.grid(True)
            ax.set_title(str(ants[k]), fontsize=14)
            if k == 0:
                ax.legend([px, py], ['X', 'Y'])
        else:
            ax.axis('off')
        if j != 0:
            ax.set_yticklabels([])
        else:
            ax.set_ylabel('amplitude', fontsize=14)
        if i != Yside-1:
            ax.set_xticklabels([])
        else:
            ax.set_xlabel('freq channel', fontsize=14)

        k += 1

## Ant Metrics

In [ ]:
antfile = xxfile_base.split('.')
antfile.pop(3)
antfile = os.path.join(data_path, '.'.join(antfile) + '.ant_metrics.json')
ant_metrics = hqm.ant_metrics.load_antenna_metrics(antfile)

In [ ]:
def plot_metric(metrics, ants=None, antpols=None, title='', ylabel='Modified z-Score', xlabel=''):
    '''Helper function for quickly plotting an individual antenna metric.'''

    if ants is None:
        ants = list(set([key[0] for key in metrics.keys()]))
    if antpols is None:
        antpols = list(set([key[1] for key in metrics.keys()]))

    for antpol in antpols:
        for i,ant in enumerate(ants):
            metric = 0
            if metrics.has_key((ant,antpol)):
                metric = metrics[(ant,antpol)]
            plt.plot(i,metric,'.')
            plt.annotate(str(ant)+antpol,xy=(i,metrics[(ant,antpol)]))
        plt.gca().set_prop_cycle(None)
    plt.title(title)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)

In [ ]:
plt.figure()
plot_metric(ant_metrics['final_mod_z_scores']['meanVij'], 
            title = 'Mean Vij Modified z-Score')

plt.figure()
plot_metric(ant_metrics['final_mod_z_scores']['redCorr'],
            title = 'Redundant Visibility Correlation Modified z-Score')

plt.figure()
plot_metric(ant_metrics['final_mod_z_scores']['meanVijXPol'], antpols=['x'],
            title = 'Modified z-score of (Vxy+Vyx)/(Vxx+Vyy)')

plt.figure()
plot_metric(ant_metrics['final_mod_z_scores']['redCorrXPol'], antpols=['x'],
            title = 'Modified z-Score of Power Correlation Ratio Cross/Same')


## Firstcal Solutions and Metrics

In [ ]:
fcfile_xx = os.path.join(data_path, xxfile_base + '.first.calfits')
fcfile_yy = os.path.join(data_path, yyfile_base + '.first.calfits')
FC_xx = hqm.firstcal_metrics.FirstCal_Metrics(fcfile_xx)
FC_yy = hqm.firstcal_metrics.FirstCal_Metrics(fcfile_yy)

FC_xx.load_metrics(fcfile_xx + '.firstcal_metrics.json')
FC_yy.load_metrics(fcfile_yy + '.firstcal_metrics.json')

In [ ]:
# print good or bad sols
print "Good firstcal solution on XX pol :",FC_xx.metrics['good_sol']
print "Good firstcal solution on YY pol :",FC_yy.metrics['good_sol']
print ""
print "Bad Ants on XX pol : ", FC_xx.metrics['bad_ants']
print "Bad Ants on YY pol : ", FC_yy.metrics['bad_ants']

In [ ]:
FC_xx.plot_delays()

In [ ]:
FC_yy.plot_delays()

In [ ]:
FC_xx.plot_stds(xaxis='ant')

In [ ]:
FC_yy.plot_stds(xaxis='ant')

## Omnical Solutions and Metrics

In [ ]:
# get files
ocfile_xx = os.path.join(data_path, xxfile_base + '.omni.calfits')
ocmodfile_xx = os.path.join(data_path, xxfile_base + '.vis.uvfits')
ocmetfile_xx = ocfile_xx + '.omni_metrics.json'
ocfile_yy = os.path.join(data_path, yyfile_base + '.omni.calfits')
ocmodfile_yy = os.path.join(data_path, yyfile_base + '.vis.uvfits')
ocmetfile_yy = ocfile_yy + '.omni_metrics.json'

# load omnical gain solutions and metrics
OM_xx = hqm.omnical_metrics.OmniCal_Metrics(ocfile_xx)
OM_yy = hqm.omnical_metrics.OmniCal_Metrics(ocfile_yy)
OM_xx.load_metrics(ocmetfile_xx)
OM_yy.load_metrics(ocmetfile_yy)
OM_xx.firstcal_delays, OM_xx.firstcal_gains = hqm.omnical_metrics.load_firstcal_gains(fcfile_xx)
OM_yy.firstcal_delays, OM_yy.firstcal_gains = hqm.omnical_metrics.load_firstcal_gains(fcfile_yy)

# load omnical model vis
uvom_xx = UVFITS()
uvom_xx.read_uvfits(ocmodfile_xx)
uvom_yy = UVFITS()
uvom_yy.read_uvfits(ocmodfile_yy)

In [ ]:
# print good or bad sols
print "Good omnical phase noise solution on XX pol :",OM_xx.metrics['phs_noise_good_sol']
print "omnical phase noise bad ants :", OM_xx.metrics['phs_noise_bad_ants']
print ""
print "Good omnical phase std solution on XX pol :",OM_xx.metrics['phs_std_good_sol']
print "omnical phase std bad ants :", OM_xx.metrics['phs_std_bad_ants']

In [ ]:
# print good or bad sols
print "Good omnical phase noise solution on YY pol :",OM_yy.metrics['phs_noise_good_sol']
print "omnical phase noise bad ants :", OM_yy.metrics['phs_noise_bad_ants']
print ""
print "Good omnical phase std solution on YY pol :",OM_yy.metrics['phs_std_good_sol']
print "omnical phase std bad ants :", OM_yy.metrics['phs_std_bad_ants']

In [ ]:
# look at XX gain solutions
OM_xx.plot_gains(divide_fc=True, plot_type='phs')
OM_xx.plot_gains(divide_fc=True, plot_type='amp')

In [ ]:
# look at YY gain solutions
OM_yy.plot_gains(divide_fc=True, plot_type='phs')
OM_yy.plot_gains(divide_fc=True, plot_type='amp')

In [ ]:
# look at xx metrics
hqm.omnical_metrics.plot_phs_metric(OM_xx.metrics, plot_type='std')
hqm.omnical_metrics.plot_phs_metric(OM_xx.metrics, plot_type='ft')

In [ ]:
# look at yy metrics
hqm.omnical_metrics.plot_phs_metric(OM_yy.metrics, plot_type='std')
hqm.omnical_metrics.plot_phs_metric(OM_yy.metrics, plot_type='ft')